In [1]:
#https://github.com/InsulatorData/InsulatorDataSet
#https://universe.roboflow.com/mingrui-yu/insulatordataset-nmard/dataset/1/download - он же.

In [2]:
%load_ext autoreload
from torchvision.datasets import VOCDetection
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import os
from pathlib import Path
import torchvision.datasets as datasets
import torchvision

import torch
import numpy as np
import cv2
from torch.utils.data import Dataset, DataLoader


In [3]:
train_dataset = VOCDetection(str(Path(os.getcwd())/'InsulatorDataSet/Normal_Insulators'),year="2007",image_set='train')
val_dataset = VOCDetection(str(Path(os.getcwd())/'InsulatorDataSet/Normal_Insulators'),year="2007",image_set='test')

In [4]:
train_dataset

Dataset VOCDetection
    Number of datapoints: 449
    Root location: /home/serhio/Data/1Education/Hack_Insulators/InsulatorDataSet/Normal_Insulators

In [5]:
# from os import listdir
# from os.path import isfile, join
# mypath = '/home/serhio/Data/1Education/Hack_Insulators/InsulatorDataSet/Normal_Insulators/VOCdevkit/VOC2007/JPEGImages'#str(Path(os.getcwd())/'InsulatorDataSet/Normal_Insulators')
# onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
# for x in onlyfiles:
#     print(x[:-4])

In [6]:
def create_model(num_classes, pretrained=False):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=pretrained)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

In [7]:
info = train_dataset[2][1]
info['annotation']['object']

[{'name': 'insulator',
  'pose': 'Unspecified',
  'truncated': '0',
  'difficult': '0',
  'bndbox': {'xmin': '131', 'ymin': '333', 'xmax': '888', 'ymax': '483'}},
 {'name': 'insulator',
  'pose': 'Unspecified',
  'truncated': '0',
  'difficult': '0',
  'bndbox': {'xmin': '374', 'ymin': '542', 'xmax': '908', 'ymax': '681'}}]

In [8]:
def pil_to_tensor(pil_image):
    return torch.from_numpy(np.asarray(pil_image)/255).permute(2, 0, 1).to(torch.float)


def info_to_dict(info):
    boxes = []
    objects_cnt = len(info['annotation']['object'])
    for object_dict in info['annotation']['object']:  # ['xmin', 'ymin', 'xmax' , 'ymax']
        # raise ValueError(object_dict)
        bbox = [
            int(object_dict['bndbox']['xmin']),
            int(object_dict['bndbox']['ymin']),
            int(object_dict['bndbox']['xmax']),
            int(object_dict['bndbox']['ymax'])
        ]
        boxes.append(bbox)
    # raise ValueError(boxes)
    boxes = torch.Tensor(boxes).to(torch.float)
    labels = torch.Tensor([1]*objects_cnt).to(torch.int64)
    return {'boxes': boxes, 'labels': labels}


info_to_dict(train_dataset[0][1])

{'boxes': tensor([[  84.,  260., 1044.,  520.]]), 'labels': tensor([1])}

In [9]:
# from PIL import Image


# def show_object_rect(image: np.ndarray, bndbox):
#     pt1 = bndbox[:2]
#     pt2 = bndbox[2:]
#     image_show = image
#     return cv2.rectangle(image_show, pt1, pt2, (0, 255, 255), 2)


# def show_object_name(image: np.ndarray, name: str, p_tl):
#     return cv2.putText(image, name, p_tl, 1, 1, (255, 0, 0))


# voc_trainset = ds
# for i, sample in enumerate(voc_trainset, 1):
#     image, annotation = sample[0], sample[1]['annotation']
#     objects = annotation['object']
#     show_image = np.array(image)
#     # print('{} object:{}'.format(i, len(objects)))
#     if not isinstance(objects, list):
#         object_name = objects['name']
#         object_bndbox = objects['bndbox']
#         x_min = int(object_bndbox['xmin'])
#         y_min = int(object_bndbox['ymin'])
#         x_max = int(object_bndbox['xmax'])
#         y_max = int(object_bndbox['ymax'])
#         show_image = show_object_rect(show_image, (x_min, y_min, x_max, y_max))
#         show_image = show_object_name(show_image, object_name, (x_min, y_min))
#     else:
#         for j in objects:
#             object_name = j['name']
#             object_bndbox = j['bndbox']
#             x_min = int(object_bndbox['xmin'])
#             y_min = int(object_bndbox['ymin'])
#             x_max = int(object_bndbox['xmax'])
#             y_max = int(object_bndbox['ymax'])
#             show_image = show_object_rect(
#                 show_image, (x_min, y_min, x_max, y_max))
#             show_image = show_object_name(
#                 show_image, object_name, (x_min, y_min))

#     # Image.fromarray(show_image).show()
#     # img = cv2.cvtColor(show_image, cv2.COLOR_BGR2RGB)
#     # im_pil = Image.fromarray(img)
#     # im_pil.show()
#     cv2.imshow('image', show_image)
#     cv2.waitKey(0)


# print(voc_trainset)
# print('Down load ok')

In [10]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = create_model(num_classes=2, pretrained=False).to(device)
#optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9, weight_decay=0.0005)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=0.0005)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)

# train_dataset = MyDataset(df_train, '/content/table-detection-dataset/images')
# val_dataset = MyDataset(df_val, '/content/table-detection-dataset/images')

train_data_loader = DataLoader(
    train_dataset,
    batch_size=3,
    shuffle=True,
    collate_fn=collate_fn
)

val_data_loader = DataLoader(
    val_dataset,
    batch_size=3,
    shuffle=False,
    collate_fn=collate_fn
)

/home/serhio/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 9010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/home/serhio/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/serhio/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current

In [12]:
def train(train_dataloader): 
    model.train()
    running_loss = 0
    for i, data in enumerate(train_dataloader):
        optimizer.zero_grad()
        images, targets = data[0], data[1]
        images = map(pil_to_tensor, images)
        targets = map(info_to_dict, targets)
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        loss_dict = model(images, targets)
        loss = sum(loss for loss in loss_dict.values())
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
        if i % 2 == 0:
            print(f"\tИтерация (батч) #{i} loss: {loss}")
    train_loss = running_loss/len(train_dataloader.dataset)
    return train_loss

def val(val_dataloader):
    running_loss = 0
    for data in val_dataloader:
        optimizer.zero_grad()
        images, targets = data[0], data[1]
        images = map(pil_to_tensor, images)
        targets = map(info_to_dict, targets)
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        with torch.no_grad():
            loss_dict = model(images, targets)
        loss_dict = model(images, targets)
        loss = sum(loss for loss in loss_dict.values())
        running_loss += loss.item()
    val_loss = running_loss/len(val_dataloader.dataset)
    return val_loss 

In [13]:
import time


train_losses = []
val_losses = []
try:
    for epoch in range(20):
        start = time.time()
        train_loss = train(train_data_loader)
        val_loss = val(val_data_loader)
        scheduler.step()
        print(f"Эпоха #{epoch} train_loss: {train_loss}, val_loss: {val_loss}")  
        end = time.time()
        print(f"Потрачено {round((end - start) / 60, 1)} минут на {epoch} эпоху")
        train_losses.append(train_loss)
        val_losses.append(val_loss)
except KeyboardInterrupt:
    print('Прервано пользователем')

	Итерация (батч) #0 loss: 1.584665298461914
Прервано пользователем
